In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model

class LayerNorm(nn.Module):
    """LayerNorm that supports both channels_last and channels_first data formats"""
    def __init__(self, normalized_shape, eps=1e-6, data_format='channels_last'):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        
        self.normalized_shape = (normalized_shape, )

    def forward(self, x):
        if self.data_format == 'channels_last':
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == 'channels_first':
            u = x.mean(1, keepdim=True)
            s = (x-u).pow(2).mean(1, keepdim=True)
            x = (x-u) / torch.sqrt(s+self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x

class Block(nn.Module):
    """ConvNeXT Block: depthwise conv, LayerNorm, pointwise conv"""
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim)  # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim)  # pointwise/1x1 convs
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones(dim), requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1)  # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)
        
        x = input + self.drop_path(x)
        return x

class ConvNeXt(nn.Module):
    """ConvNeXT backbone for UNet"""
    def __init__(self, in_chans=3, depths=[3, 3, 9, 3], 
                 dims=[96, 192, 384, 768], drop_path_rate=0.,
                 layer_scale_init_value=1e-6, out_indices=[0, 1, 2, 3]):
        super().__init__()
        self.out_indices = out_indices
        
        # Stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        
        self.downsample_layers = nn.ModuleList([stem])
        
        for i in range(3):
            downsample_layer = nn.Sequential(
                LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)
        
        # 4 feature resolution stages, each consisting of multiple residual blocks
        self.stages = nn.ModuleList()
        dp_rates = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j], 
                       layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]
        
        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
    
    def forward_features(self, x):
        features = []
        
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
            if i in self.out_indices:
                features.append(x)
        
        return features
    
    def forward(self, x):
        return self.forward_features(x)

class UpBlock(nn.Module):
    """Upsampling block for UNet decoder"""
    def __init__(self, in_channels, out_channels, skip_channels=0):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv = nn.Sequential(
            nn.Conv2d(out_channels + skip_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x, skip=None):
        x = self.up(x)
        if skip is not None:
            # Handle potential size mismatch
            diffY = skip.size()[2] - x.size()[2]
            diffX = skip.size()[3] - x.size()[3]
            
            x = F.pad(x, [diffX // 2, diffX - diffX // 2,
                          diffY // 2, diffY - diffY // 2])
            
            x = torch.cat([skip, x], dim=1)
        
        return self.conv(x)

class ConvNeXtUNet(nn.Module):
    """UNet with ConvNeXT backbone for image segmentation"""
    def __init__(self, in_channels=3, num_classes=1, 
                 backbone_depths=[3, 3, 9, 3],
                 backbone_dims=[96, 192, 384, 768],
                 drop_path_rate=0.1,
                 layer_scale_init_value=1e-6):
        super().__init__()
        
        # ConvNeXT backbone
        self.backbone = ConvNeXt(
            in_chans=in_channels,
            depths=backbone_depths,
            dims=backbone_dims,
            drop_path_rate=drop_path_rate,
            layer_scale_init_value=layer_scale_init_value
        )
        
        # Decoder
        self.up1 = UpBlock(backbone_dims[3], backbone_dims[2], backbone_dims[2])
        self.up2 = UpBlock(backbone_dims[2], backbone_dims[1], backbone_dims[1])
        self.up3 = UpBlock(backbone_dims[1], backbone_dims[0], backbone_dims[0])
        
        # Final upsampling and output
        self.up_final = nn.ConvTranspose2d(backbone_dims[0], backbone_dims[0] // 2, kernel_size=2, stride=2)
        self.final_conv = nn.Conv2d(backbone_dims[0] // 2, backbone_dims[0] // 4, kernel_size=3, padding=1)
        self.final_up = nn.ConvTranspose2d(backbone_dims[0] // 4, backbone_dims[0] // 8, kernel_size=2, stride=2)
        self.final = nn.Conv2d(backbone_dims[0] // 8, num_classes, kernel_size=1)
        
    def forward(self, x):
        # Get features from backbone
        features = self.backbone(x)
        
        # Decoder path
        x = self.up1(features[3], features[2])
        x = self.up2(x, features[1])
        x = self.up3(x, features[0])
        
        # Final layers
        x = self.up_final(x)
        x = F.relu(self.final_conv(x))
        x = self.final_up(x)
        x = self.final(x)
        
        return x

# Example of how to create a ConvNeXtUNet with different configurations
def convnext_unet_tiny(in_channels=3, num_classes=1):
    return ConvNeXtUNet(
        in_channels=in_channels,
        num_classes=num_classes,
        backbone_depths=[3, 3, 9, 3],
        backbone_dims=[96, 192, 384, 768]
    )

def convnext_unet_small(in_channels=3, num_classes=1):
    return ConvNeXtUNet(
        in_channels=in_channels,
        num_classes=num_classes,
        backbone_depths=[3, 3, 27, 3],
        backbone_dims=[96, 192, 384, 768]
    )

def convnext_unet_base(in_channels=3, num_classes=1):
    return ConvNeXtUNet(
        in_channels=in_channels,
        num_classes=num_classes,
        backbone_depths=[3, 3, 27, 3],
        backbone_dims=[128, 256, 512, 1024]
    )

def convnext_unet_large(in_channels=3, num_classes=1):
    return ConvNeXtUNet(
        in_channels=in_channels,
        num_classes=num_classes,
        backbone_depths=[3, 3, 27, 3],
        backbone_dims=[192, 384, 768, 1536]
    )

# Example usage
model = convnext_unet_large(num_classes=1)
x = torch.randn(1, 3, 256, 256)
output = model(x)
print(output.shape)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

torch.Size([1, 1, 256, 256])
Input shape: torch.Size([1, 3, 256, 256])
Output shape: torch.Size([1, 1, 256, 256])
